##Atividade desafio pandas

Pegar a base de dados da Alesp(assembleia legislativa de são paulo) e realizar os tratamentos necessários com as seguintes diretrizes:
- Utilizar obrigatoriamente pandas 
- Extrair diretamente do site o arquivo XML dos gastos e cadastros de cada deputado
- Verificar e corrigir possíveis inconsistências nos dados que podem ser (Nomes diferentes para o mesmo item ex: TÁXI e taxi ) ou valores ausentes ou duplicidade de dados
- verificar e contar os valores ausentes (NaN , NA , etc)
- realizar insights onde mostrem os valores totais dos ultimos 4 anos dos valores gastos por cada deputado
- mostrar os itens dos ultimos quatro anos onde se mais teve despesa 
- mostrar os 3 deputados que mais e menos gastam
- realizar a conversão do dataframe para CSV e disponibiliza-lo em um bucket com acesso público
- Os insights não se limitam aos pedidos acima , onde gostaríamos de pelo menos mais 2 
- Os resultados podem ou não trazer plotagem(trate como não obrigatório ) mas é recomendado que traga
- Montar uma pequena apresentação no próprio notebook trazendo uma organização e fluxo de ETL

Integrantes: Jum Saheki, Ricardo Corsini


##INSTALL & IMPORTS

In [ ]:
# gcsfs para criar conexão com a Google Cloud Platform e o Google drive
!pip install gcsfs

In [363]:
# Bibliotecas do google
from google.cloud import storage
from google.colab import drive
import os

In [ ]:
# lib para conexão
import requests

# Modelagem de dados
import pandas as pd

# Ignorando avisos
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Pandera para fazer a verificação dos dados
!pip install pandera

In [358]:
import pandera as pa
from pandera.checks import Check

##CONEXÃO COM GCP E GOOGLE DRIVE

In [364]:
# Estabelecendo conexão com o google drive para acessar a chave do GCP
drive.mount('/content/drive')

Mounted at /content/drive


In [365]:
# Configuração da chave de segurança
serviceAccount = '/content/drive/MyDrive/Soul Code/macro-mercury-349020-d9ed9a670580.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [366]:
# Acesso ao bucket
client = storage.Client()
bucket = client.get_bucket('soulcode')

## OBTENDO OS DADOS

In [367]:
# URL da API da Alesp
URL_Despesas = 'http://www.al.sp.gov.br/repositorioDados/deputados/despesas_gabinetes.xml'
URL_Cadastro = 'https://www.al.sp.gov.br/repositorioDados/deputados/deputados.xml'

In [368]:
# Pegando os dados
Conexao_01 = requests.get( URL_Despesas )
Conexao_02 = requests.get( URL_Cadastro )

# Verificando se a API funcinou, o número 200 representa sucesso
print( Conexao_01, Conexao_02 )

<Response [200]> <Response [200]>


In [369]:
# Lendo arquivo xml
df = pd.read_xml(Conexao_01.content)


In [379]:
# Backup do DataFrame
df2 = df.copy()

##PRÉ ANÁLISE


In [381]:
# Removendo valores anteriores a 4 anos
df2 = df2[(df2.Ano.isin([2019,2020,2021,2022]))]

In [382]:
# Removendo coluna Matrícula, pois ela serve apenas para identificar o deputado
df2 = df2.drop(columns=['Matricula'])

In [ ]:
# Número de ocorrencias de linhas duplicadas
df2.duplicated().sum()

In [383]:
# Número de ocorrencia de valores nulos em cada coluna
df2.isna().sum()

Ano             0
Mes             0
Valor           0
CNPJ          825
Deputado        0
Tipo            0
Fornecedor      0
dtype: int64

In [384]:
# Determinando quais são os registros nulos na coluna CNPJ
invalidos = df2.CNPJ.isnull()
ausente = df2.loc[invalidos]
pd.unique(ausente['Fornecedor'])

Observa-se que os valores 'PEDÁGIO', 'PEDAGIO', 'PEDÁGIOS', 'CUPONS DE PEDÁGIO' se referem ao mesmo fornecedor


In [ ]:
# Analisando os dados do fornecedor TAXI
df2[df2['Fornecedor'] == 'TAXI']

In [ ]:
# Quantos fornecedores TAXI possuem CNPJ nulos
df2[df2['Fornecedor'] == 'TAXI'].isna().sum()

Todos os fornecedores do tipo 'TAXI' possuem CNPJ 'None'

In [ ]:
# Analisando os CNPJs cujo fornecedor é do tipo Pedágio
df2[df2['Fornecedor'] == 'PEDÁGIO']

In [ ]:
df2[df2['Fornecedor'] == 'PEDÁGIO'].isna().sum()

In [ ]:
df2[(df2['Fornecedor'] == 'PEDÁGIO') & (~df2['CNPJ'].isnull())]

In [ ]:
df2[df2['Fornecedor'] == 'PEDAGIO']

In [ ]:
df2[df2['Fornecedor'] == 'PEDÁGIOS']

In [ ]:
df2[df2['Fornecedor'] == 'CUPONS DE PEDÁGIO']

Os pedágios possuem CNPJ nulos, 0000000000000, 02509491000126, 02415408000150

In [ ]:
# Verificando tipo de dados de cada coluna
df2.dtypes

##ANÁLISE

Procurando por valores inconsistentes

In [ ]:
sorted(pd.unique(df2['Ano']))

In [ ]:
sorted(pd.unique(df2['Mes']))

In [ ]:
sorted(pd.unique(df2['Valor']))

In [ ]:
sorted(pd.unique(df2['Valor']),reverse=True)

As colunas com valores numéricos não apresentam inconsistências

In [ ]:
pd.unique(df2['CNPJ'])

In [ ]:
pd.unique(df2['Deputado'])

In [ ]:
sorted(pd.unique(df2['Tipo']))

In [ ]:
sorted(pd.unique(df2['Fornecedor']))

Na coluna Fornecedor há múltiplos valores semelhantes que possuem o mesmo CNPJ

##TRATAMENTO

In [389]:
 #Determinando um CNPJ para fornecedor do tipo 'TAXI'
df2.loc[df2['Fornecedor'] == 'TAXI', 'CNPJ'] = 1

In [390]:
#Determinado um CNPJ para fornecedor do tipo 'PEDÁGIO', 'PEDAGIO', 'PEDÁGIOS', 'CUPONS DE PEDÁGIO'
df2.loc[df2['Fornecedor'] == 'PEDÁGIO', 'CNPJ'] = 2
df2.loc[df2['Fornecedor'] == 'PEDAGIO', 'CNPJ'] = 2
df2.loc[df2['Fornecedor'] == 'PEDÁGIOS', 'CNPJ'] = 2
df2.loc[df2['Fornecedor'] == 'CUPONS DE PEDÁGIO', 'CNPJ'] = 2

In [ ]:
# Verificando os valores nulos
df2.isna().sum()

In [392]:
#Convertendo a coluna 'CNPJ' para o tipo int
df2['CNPJ'] = df2['CNPJ'].astype(int)

## VERIFICAÇÃO

In [ ]:
df2.dtypes

In [398]:
schema = pa.DataFrameSchema( 
      columns= {
        'Ano':pa.Column(pa.Int),
        'Mes':pa.Column(pa.Int),
        'Valor':pa.Column(pa.Float64),
        'CNPJ':pa.Column(pa.Int),
        'Deputado':pa.Column(pa.String),
        'Tipo':pa.Column(pa.String),
        'Fornecedor':pa.Column(pa.String)
      }
)

In [ ]:
schema.validate(df2)

##INSIGHTS


mostrar os 3 deputados que mais e menos gastam

In [ ]:
dep_mais = df2.groupby(['Deputado']).Valor.sum().sort_values(ascending=False).head(3)
dep_mais

In [ ]:
dep_mais.plot.bar(xlabel="Deputados",ylabel="Gastos",title="Deputados que mais gastam",grid=True,color=['y','black','purple'])

In [ ]:
dep_menos = df2.groupby(['Deputado']).Valor.sum().sort_values().head(3)
dep_menos

In [ ]:
dep_menos.plot.bar(xlabel="Deputados",ylabel="Gastos",title="Deputados que menos gastam",color=['r','g','b'],grid=True)

Mostrar os itens dos ultimos quatro anos onde se mais teve despesa

In [ ]:
cnpj_mais = df2.groupby(['CNPJ']).Valor.sum().sort_values(ascending=False).head(10)
cnpj_mais

In [ ]:
cnpj_mais.plot.bar(xlabel="Deputados",ylabel="Gastos",title="Itens mais utilizados",grid=True, color=['g','b','black','yellow','r'])

Itens com menos despesas

In [ ]:
cnpj_menos = df2.groupby(['CNPJ','Fornecedor']).Valor.sum().sort_values(ascending=False).head(10)
cnpj_menos

In [ ]:
cnpj_menos.plot.bar(xlabel="Deputados",ylabel="Gastos",title="Itens menos utilizados",grid=True, color=['r','g','b','y'])

Progressão de gastos de ano a ano

In [ ]:
ano_gastos = df2.groupby('Ano').Valor.sum()
ano_gastos

In [ ]:
ano_gastos.plot.line(xlabel="Ano",ylabel="Gastos",title="Progressão de gastos",grid=True)

Gastos por tipo

In [ ]:
tipo_gasto = df2.groupby('Tipo').Valor.sum().sort_values()
tipo_gasto

In [ ]:
tipo_gasto.plot(xlabel='Tipo',ylabel='Gastos',title='Tipo mais gastos', kind='barh',grid=True)

## ENVIANDO O DATAFRAME EM FORMATO CSV PARA A BUCKET NO GCP

In [400]:
df2.to_csv('gs://soulcode/Tratado/alesp.csv')